<a href="https://colab.research.google.com/github/chaitanyap1/Datascience-projects/blob/master/Quora_questions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np

from keras.models import Sequential
#from keras.layers.recurrent import LSTM,GRU
#from keras.layers.core import Dense, Activation, Dropout
from sklearn.metrics import roc_auc_score, accuracy_score
#from sklearn.utils import shuffle
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras import models
from keras.models import load_model

from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.layers import LSTM, Activation, Dense, Input, Embedding, Dropout
from keras.models import Model
from nltk import word_tokenize

from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight

Using TensorFlow backend.


In [0]:
!wget https://www.dropbox.com/sh/kpf9z73woodfssv/AAAwZ5DDt-aHwqZFHYrZ-ZBHa/train.csv?dl=0

--2020-01-10 13:01:44--  https://www.dropbox.com/sh/kpf9z73woodfssv/AAAwZ5DDt-aHwqZFHYrZ-ZBHa/train.csv?dl=0
Resolving www.dropbox.com (www.dropbox.com)... 162.125.8.1, 2620:100:601b:1::a27d:801
Connecting to www.dropbox.com (www.dropbox.com)|162.125.8.1|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /sh/raw/kpf9z73woodfssv/AAAwZ5DDt-aHwqZFHYrZ-ZBHa/train.csv [following]
--2020-01-10 13:01:45--  https://www.dropbox.com/sh/raw/kpf9z73woodfssv/AAAwZ5DDt-aHwqZFHYrZ-ZBHa/train.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucd8bd8a26cd94e732b819e3c955.dl.dropboxusercontent.com/cd/0/inline/Av7kce7xJ4sKfHjFw0fZ3Pnnnk_rLm-llopYq5qRpinGXpm15J2s88TnVOeA-kGooepj9Q1FP5RGpdTmxRZri5jGYnjnI_rdvIRfQJIDgTHOIQ/file# [following]
--2020-01-10 13:01:45--  https://ucd8bd8a26cd94e732b819e3c955.dl.dropboxusercontent.com/cd/0/inline/Av7kce7xJ4sKfHjFw0fZ3Pnnnk_rLm-llopYq5qRpinGXpm15J2s88TnVO

In [0]:
!wget http://nlp.stanford.edu/data/glove.42B.300d.zip

--2020-01-10 13:01:53--  http://nlp.stanford.edu/data/glove.42B.300d.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.42B.300d.zip [following]
--2020-01-10 13:01:53--  https://nlp.stanford.edu/data/glove.42B.300d.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip [following]
--2020-01-10 13:01:53--  http://downloads.cs.stanford.edu/nlp/data/glove.42B.300d.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1877800501 (1.7G) [application/zip]
Sav

In [0]:
!unzip /content/glove.42B.300d.zip

Archive:  /content/glove.42B.300d.zip
  inflating: glove.42B.300d.txt      


In [0]:
embedding_index ={}

f = open('/content/glove.42B.300d.txt', encoding='utf-8')
for line in f:
  values = line.split()
  word = values[0]
  coefs = np.asarray(values[1:], dtype ='float32')
  embedding_index[word] = coefs
f.close()

In [0]:
question = pd.read_csv(r'/content/train.csv?dl=0')

In [0]:
print(question.shape)
question.head()

(1306122, 3)


,qid,question_text,target
0,00002165364db923c7e6,How did Quebec nationalists see their province...,0
1,000032939017120e6e44,"Do you have an adopted dog, how would you enco...",0
2,0000412ca6e4628ce2cf,Why does velocity affect time? Does velocity a...,0
3,000042bf85aa498cd78e,How did Otto von Guericke used the Magdeburg h...,0
4,0000455dfa3e01eae3af,Can I convert montra helicon D to a mountain b...,0


In [0]:
X = question['question_text']
Y = question['target']
x_train, x_test, y_train, y_test = train_test_split(X,Y,test_size=0.2)
x_train.shape, y_train.shape, x_test.shape, y_test.shape

((1044897,), (1044897,), (261225,), (261225,))

In [0]:
y_test.value_counts()

0    244846
1     16379
Name: target, dtype: int64

In [0]:
class_weight = class_weight.compute_class_weight('balanced', 
                                                 np.unique(y_train), 
                                                    y_train)
print(class_weight)
class_weight_dict = dict(enumerate(class_weight))
class_weight_dict

[0.53285734 8.10865112]


{0: 0.5328573351855138, 1: 8.108651115146436}

In [0]:
x_train

994698     Is it advisable to join a startup even if your...
322285     What was the cut off for BBI in Mithibai colle...
845300     Can I do civil engineering from GB Pant via th...
1156252           How we make questions about animal sounds?
1098107    Thick is the best country to get a facelift at...
                                 ...                        
751946     What were the most important results of the Se...
1274427      How can I write a biography about Robert Coles?
1300533    Some days I can keep going pretty good but oth...
955649     How do I prepare for NCHM JEE 2018 in about 30...
888135     Can I take a new subject at A levels in class 12?
Name: question_text, Length: 1044897, dtype: object

In [0]:
import nltk
nltk.download('punkt')  

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [0]:
sent_lens=[len(word_tokenize(x)) for x in x_train]
print(max(sent_lens))

412


In [0]:
print(np.percentile(sent_lens,99))
np.percentile(sent_lens,95)

45.0


31.0

In [0]:
max_len=45

tk=Tokenizer(char_level=False,split=' ')
tk.fit_on_texts(x_train)
print(x_train.shape)

(1044897,)


In [0]:
x_train

1084293                            What is the value of √36?
520714     What was the most memorable random act of kind...
971229     If I have visitor visas of US and Canada havin...
760502                              How is tea tasting done?
1181263    Is it normal that your dad watch porn even if ...
                                 ...                        
1098172    What are the chances of an English Honours gra...
205719     Whatis the smallest number that can be divided...
183517               Which is the best course after +2 exam?
327888                                    Is Buddhism dying?
469996                                  How is Mi A1 better?
Name: question_text, Length: 1044897, dtype: object

In [0]:
seq_train=tk.texts_to_sequences(x_train)
seq_test=tk.texts_to_sequences(x_test)
print(len(seq_test), len(seq_train))
seq_train[0]

261225 1044897


[3,
 17,
 4019,
 5,
 371,
 4,
 880,
 147,
 20,
 28,
 137,
 26,
 44,
 24,
 4,
 53,
 340,
 13,
 17,
 57,
 26,
 44,
 24,
 4,
 53,
 340,
 13,
 4771]

In [0]:
vocab_size=len(tk.word_index)
vocab_size

195943

In [0]:
seq_train_matrix=sequence.pad_sequences(seq_train,maxlen=max_len)
seq_test_matrix=sequence.pad_sequences(seq_test,maxlen=max_len)
seq_train_matrix.shape, seq_test_matrix.shape

((1044897, 45), (261225, 45))

In [0]:
# creating our own embedding matrix to bring down the size to 300
# we'll use 300 D vector representation of the words from pretrained embedding index 
# that we downloaded 

embedding_matrix=np.zeros((vocab_size+1,300))

for word,i in tk.word_index.items():
    embed_vector=embedding_index.get(word)
    if embed_vector is not None:
        embedding_matrix[i]=embed_vector
        
# if there are specific words which are not present in pretrained embedding 
# their weights will remain 0. if there are too many such words 
# then you should probably not use pretrained embeddings 


In [0]:
# Now while creating the model for text data , we'll keep embedding layer weights 
# non-trainable as we are supplying those values from our end 

inputs = Input(name='text_input',shape=[max_len])
embed=Embedding(vocab_size+1,300,input_length=max_len,mask_zero=True,
                weights=[embedding_matrix],trainable=False)(inputs)
lstm_layer1 = LSTM(512)(embed)
drop1 = Dropout(0.2)(lstm_layer1)
lstm_layer2 = Dense(256,activation='relu')(drop1)
drop2 = Dropout(0.2)(lstm_layer2)
lstm_layer3 = Dense(128,activation='relu')(drop2)
drop3 = Dropout(0.2)(lstm_layer3)
dense1 = Dense(25,activation='relu')(lstm_layer3)
drop4 = Dropout(0.2)(dense1)
final_layer = Dense(1,activation='sigmoid')(drop4)

model = Model(inputs=inputs,outputs=final_layer)










Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [0]:
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_input (InputLayer)      (None, 45)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 45, 300)           58854000  
_________________________________________________________________
lstm_1 (LSTM)                (None, 512)               1665024   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               3289

In [0]:
filepath='/content/weights-{epoch:02d}-{val_loss:.4f}.h5'
checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, 
                             save_best_only=True)

In [0]:
earlystop = EarlyStopping(monitor='val_loss', patience=3,
                          verbose=1)

In [0]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [0]:
model.fit(seq_train_matrix,y_train,
              epochs=10,
              class_weight={0: 0.5328573351855138, 1: 8.108651115146436},
              batch_size=10000,
              validation_data=(seq_test_matrix,y_test),
              callbacks = [earlystop, checkpoint])

Train on 1044897 samples, validate on 261225 samples
Epoch 1/10
1044897/1044897 [==============================] - 804s 770us/step - loss: 0.2981 - acc: 0.8721 - val_loss: 0.2338 - val_acc: 0.9008

Epoch 00001: val_loss improved from inf to 0.23376, saving model to /content/weights-01-0.2338.h5
Epoch 2/10
1044897/1044897 [==============================] - 793s 759us/step - loss: 0.2650 - acc: 0.8868 - val_loss: 0.2454 - val_acc: 0.8959

Epoch 00002: val_loss did not improve from 0.23376
Epoch 3/10
1044897/1044897 [==============================] - 797s 763us/step - loss: 0.2508 - acc: 0.8918 - val_loss: 0.2507 - val_acc: 0.8976

Epoch 00003: val_loss did not improve from 0.23376
Epoch 4/10
1044897/1044897 [==============================] - 807s 772us/step - loss: 0.2373 - acc: 0.8954 - val_loss: 0.2715 - val_acc: 0.8836

Epoch 00004: val_loss did not improve from 0.23376
Epoch 00004: early stopping


In [0]:
best_model = load_model('/content/weights-01-0.2338.h5')

In [0]:
best_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
text_input (InputLayer)      (None, 45)                0         
_________________________________________________________________
embedding_1 (Embedding)      (None, 45, 300)           58783200  
_________________________________________________________________
lstm_1 (LSTM)                (None, 512)               1665024   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_2 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               3289

In [0]:
p = best_model.predict(seq_test_matrix)
roc_auc_score(y_test,p)

0.9528024281838164

In [0]:
p

array([[0.39858234],
       [0.11947393],
       [0.6753924 ],
       ...,
       [0.01505166],
       [0.01708287],
       [0.29535776]], dtype=float32)